In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        (x_train, y_train), (x_valid, y_valid), _ = pickle.load(f, encoding='latin-1')
    return map(tensor, [x_train, y_train, x_valid, y_valid])
#

def normalize(x, m, s): return (x - m) / s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

(torch.Size([50000, 784]),
 torch.Size([50000]),
 torch.Size([10000, 784]),
 torch.Size([10000]))

In [5]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [6]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
train_mean,train_std = x_train.mean(), x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [8]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs() < tol 

In [9]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [10]:
n,m = x_train.shape
c = y_valid.max() + 1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [11]:
# num hidden
nh = 50

In [12]:
# simplified kaiming init / he init
w1 = torch.randn(m, nh) / math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1) / math.sqrt(nh)
b2 = torch.zeros(1)

In [13]:
test_near_zero(w1.mean())
test_near_zero(w1.std() - 1/math.sqrt(m))

In [14]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [15]:
def lin(x, w, b): return x@w + b

In [16]:
t = lin(x_valid, w1, b1)

In [17]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(-0.0713), tensor(0.9584))

In [18]:
torch.tensor([3, -1, 6, -9, 8]).clamp_min(0)

tensor([3, 0, 6, 0, 8])

In [19]:
def relu(x): return x.clamp_min(0)

In [20]:
t = relu(lin(x_valid, w1, b1))

In [21]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3442), tensor(0.5255))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [22]:
# kaiming init / he init for relu
w1 = torch.randn(m, nh) * math.sqrt(2 / m)

In [23]:
w1.mean(),w1.std()

(tensor(0.0005), tensor(0.0504))

In [24]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5324), tensor(0.7889))

In [25]:
#export
from torch.nn import init

In [26]:
w1 = torch.zeros(m,nh)
init.kaiming_uniform_(w1)
t = relu(lin(x_valid, w1, b1))

In [27]:
w1.mean(),w1.std()

(tensor(-0.0008), tensor(0.2008))

In [28]:
t.mean(),t.std()

(tensor(2.2051), tensor(3.3026))

In [29]:
w1.shape

torch.Size([784, 50])

In [30]:
import torch.nn

In [31]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

In [32]:
torch.nn.Linear.forward??

In [33]:
torch.nn.functional.linear??

In [ ]:
torch.nn.Conv2d??

In [36]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [33]:
# what if...?
def relu(x): return x.clamp_min(0) - 0.5

In [34]:
# kaiming init / he init for relu
w1 = torch.randn(m, nh) * math.sqrt(2 / m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0468), tensor(0.8151))

In [35]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [36]:
%timeit -n 10 _=model(x_valid)

4.51 ms ± 1.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

In [38]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [39]:
#export
def mse(output, targ): return (output - targ[:,None]).pow(2).mean(dim=0)

In [40]:
y_train,y_valid = y_train.float(), y_valid.float()

In [41]:
preds = model(x_train)

In [42]:
mse(preds, y_train)

tensor([25.0558])

### Gradients and backward pass

In [43]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2 * (inp - targ[:,None]) / len(inp)

In [44]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = out.g * (inp > 0).float()

In [45]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = inp.t() @ out.g
    b.g = out.g.sum(dim=0)

In [46]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [47]:
forward_and_backward(x_train, y_train)

In [48]:
# Save for testing against later
w1g = w1.g
w2g = w2.g
b1g = b1.g
b2g = b2.g
ig  = x_train.g

We cheat a little bit and use PyTorch autograd to check our results.

In [49]:
xt2 = x_train.clone().requires_grad_()
w12 = w1.clone().requires_grad_()
w22 = w2.clone().requires_grad_()
b12 = b1.clone().requires_grad_()
b22 = b2.clone().requires_grad_()

In [50]:
def forward(inp, targ):
    # forward pass:
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [51]:
loss = forward(xt2, y_train)

In [52]:
loss.backward()

In [53]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig)

## Refactor model

### Layers as classes

In [54]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = self.inp.clamp_min(0) - 0.5
        return self.out
    
    def backward(self): self.inp.g = self.out.g * (self.inp > 0).float()

In [55]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = self.inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        #self.w.g = inp.t() @ out.g
        self.w.g = (self.inp[:,:,None] * self.out.g[:,None,:]).sum(dim=0)
        self.b.g = self.out.g.sum(dim=0)

In [56]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (self.inp - self.targ[:,None]).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2 * (self.inp - self.targ[:,None]) / len(self.inp)

In [57]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.lin1, self.relu, self.lin2 = Lin(w1, b1), Relu(), Lin(w2, b2)
        self.loss = Mse()
        
    def __call__(self, inp, targ):
        self.out = self.lin2(self.relu(self.lin1(inp)))
        return self.loss(self.out, targ)
    
    def backward(self):
        for l in [self.loss, self.lin2, self.relu, self.lin1]:
            l.backward()

In [58]:
w1.g, b1.g, w2.g, b2.g, x_train.g = 5 * [None]
model = Model(w1, b1, w2, b2)

In [59]:
%time loss = model(x_train, y_train)

CPU times: user 136 ms, sys: 4 ms, total: 140 ms
Wall time: 37.6 ms


In [60]:
%time model.backward()

CPU times: user 3.81 s, sys: 3.6 s, total: 7.41 s
Wall time: 6.47 s


In [61]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [75]:
class Module():
    def __call__(self, inp, *args):
        if args: self.args = args
        self.inp, self.out = inp, self.forward(inp, *args)
        return self.out
    
    def forward(self): raise Exception("Not implemented")
    def backward(self): self.bwd()

In [76]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0) - 0.5
    def bwd(self): self.inp.g = self.out.g * (self.inp > 0).float()

In [77]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
        
    def forward(self, inp): return inp @ self.w + self.b
    
    def bwd(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp[:,:,None] * self.out.g[:,None,:]).sum(dim=0)
        self.b.g = self.out.g.sum(dim=0)

In [78]:
class Mse(Module):
    def forward(self, inp, targ): return (inp - targ[:,None]).pow(2).mean()
    def bwd(self): self.inp.g = 2 * (self.inp - self.args[0][:,None]) / len(self.inp)

In [79]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, inp, targ):
        for l in self.layers: inp = l(inp)
        return self.loss(inp, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [80]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [81]:
%time loss = model(x_train, y_train)

CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 27.3 ms


In [82]:
%time model.backward()

CPU times: user 3.74 s, sys: 3.13 s, total: 6.87 s
Wall time: 5.94 s


In [83]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [84]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
        
    def forward(self, inp): return inp @ self.w + self.b
    
    def bwd(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(dim=0)

In [85]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [86]:
%time loss = model(x_train, y_train)

CPU times: user 88 ms, sys: 0 ns, total: 88 ms
Wall time: 24.6 ms


In [87]:
%time model.backward()

CPU times: user 480 ms, sys: 64 ms, total: 544 ms
Wall time: 139 ms


In [88]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [89]:
#export
from torch import nn

In [93]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.l1, self.relu, self.l2 = nn.Linear(n_in, nh), Relu(), nn.Linear(nh, n_out)
        self.loss = Mse()
        
    def __call__(self, inp, targ):
        out = self.l2(self.relu(self.l1(inp)))
        return self.loss(out, targ)

In [94]:
model = Model(m, nh, 1)

In [95]:
%time loss = model(x_train, y_train)

CPU times: user 120 ms, sys: 12 ms, total: 132 ms
Wall time: 41 ms


In [96]:
%time loss.backward()

CPU times: user 148 ms, sys: 4 ms, total: 152 ms
Wall time: 56.8 ms


## Export

In [ ]:
!python notebook2script.py 02_fully_connected.ipynb